# Hyperparameter selection with Ray Tune

experiment_tag: accum_iter=8,batch_size=16,is_amsgrad=False,lr=0.0012185,middle_fc=False,middle_fc_size=0,weight_decay=0.000125

In [1]:
import os
from datetime import datetime
import time
import random
import warnings
import joblib
import warnings
import gc
# My modules
from config import Config
from logger import init_logger
from common_utils import set_seeds, read_csvs, stratify_split, setup, get_data_dfs, get_loaders
from model import Model
from train_loop_functions import train_epoch, valid_epoch

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score

import torch
from torch import nn
from torch.cuda.amp import GradScaler

# hyperparameter tuning
from functools import partial
import ray
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler
from ray.tune.suggest.ax import AxSearch
from ray.tune.suggest.bayesopt import BayesOptSearch

In [2]:
%load_ext autoreload
%autoreload 2

# Setup

In [3]:
set_seeds(Config.seed)
LOGGER = init_logger() # uses Python's logging framework

# Tuning

In [4]:
# all params are passed in by Tune. this is the "objective" function.
def train_main(config, checkpoint_dir=None, data_dir=None):
    assert config is not None
    # -------- DATASETS AND LOADERS --------
    data_df, _, _ = read_csvs(data_dir, Config.debug, num_samples=10000)
    train_folds = stratify_split(data_df, Config.fold_num, Config.seed, Config.target_col)
    
    # select only one of the folds (fold 0)
    train_df, valid_df = get_data_dfs(train_folds, 0)
    train_dataloader, valid_dataloader = get_loaders(train_df, valid_df,
                                                     config["batch_size"], 
                                                     data_dir+'/train_images')
    
    # -------- MODEL --------
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model, optimizer = setup_model_optimizer(Config.model_arch, 
                                                   config["lr"], 
                                                   config["is_amsgrad"], 
                                                   num_labels=data_df.label.nunique(), 
                                                   fc_layer={
                                                       'middle_fc': config["middle_fc"], 
                                                        'middle_fc_size': config["middle_fc_size"]
                                                    }, 
                                                   weight_decay=config["weight_decay"],
                                                   device=device,
                                                   checkpoint=None)
    
    scheduler, criterion = get_schd_crit()
    
    # The `checkpoint_dir` parameter gets passed by Ray Tune when a checkpoint should be restored.
    if checkpoint_dir:
        checkpoint = os.path.join(checkpoint_dir, "checkpoint")
        model_state, optimizer_state = torch.load(checkpoint)
        model.load_state_dict(model_state)
        optimizer.load_state_dict(optimizer_state)
        
    # EPOCHS TRAIN
    for e in range(10):
        # TRAIN
        avg_training_loss = train_epoch(train_dataloader, model, 
                                      criterion, optimizer, 
                                      scheduler, GradScaler(), 
                                      config["accum_iter"], LOGGER,
                                      device)

        # VALIDATE
        avg_validation_loss, preds = valid_epoch(valid_dataloader, model, criterion, LOGGER, device)
  
        validation_labels = valid_df[Config.target_col].values
        accuracy = accuracy_score(y_true=validation_labels, y_pred=preds)
        
        # SAVE CHECKPOINT.
        # It is automatically registered with Ray Tune and will potentially
        # be passed as the `checkpoint_dir` parameter in future iterations.
        with tune.checkpoint_dir(step=e) as checkpoint_dir:
            path = os.path.join(checkpoint_dir, "checkpoint")
            torch.save((model.state_dict(), optimizer.state_dict()), path)

        tune.report(loss=avg_validation_loss, accuracy=accuracy)
        gc.collect()

    print("Finished Training")

In [5]:
def main(num_samples=50, max_num_epochs=10, gpus_per_trial=1):
    data_dir = os.path.abspath('./data')
    
    hyperconfig = {
        "is_amsgrad": False,
        "accum_iter": tune.choice([2,4,8]),
        "lr": tune.loguniform(1e-3, 1e-1),
        "batch_size": tune.choice([8, 16]),
        "weight_decay": tune.choice([1/8 * 0.001, 1/2 * 0.001, 0.]),
        # try adding a FC layer to the classifier portion of the model
        "middle_fc": False,
        "middle_fc_size": 0
    }
    
    scheduler = ASHAScheduler(
        #metric="loss",
        mode="min",
        max_t=max_num_epochs,
        grace_period=2,
        reduction_factor=2)
    reporter = CLIReporter(metric_columns=["loss", "accuracy", "training_iteration"])
    
    search_alg = AxSearch(metric="loss", mode="min")
    #search_alg = BayesOptSearch(metric="loss", mode="min")
    result = tune.run(
        partial(train_main, data_dir=data_dir),
        name="ax3",
        metric="loss",
        resources_per_trial={"cpu": os.cpu_count(), "gpu": gpus_per_trial},
        config=hyperconfig,
        num_samples=num_samples,
        scheduler=scheduler,
        progress_reporter=reporter,
        search_alg=search_alg,
        local_dir='./ray-results',
        checkpoint_score_attr='min-loss'
    )

    best_trial = result.get_best_trial("loss", "min", "last")
    print("Best trial config: {}".format(best_trial.config))
    print("Best trial final validation loss: {}".format(
        best_trial.last_result["loss"]))
    print("Best trial final validation accuracy: {}".format(
        best_trial.last_result["accuracy"]))

In [ ]:
main()

2020-12-27 05:08:29,141	INFO services.py:1092 -- View the Ray dashboard at http://127.0.0.1:8265
2020-12-27 05:08:29,765	WARNING experiment.py:274 -- No name detected on trainable. Using DEFAULT.
2020-12-27 05:08:29,766	INFO registry.py:65 -- Detected unknown callable for trainable. Converting to class.
[INFO 12-27 05:08:29] ax.service.ax_client: Starting optimization with verbose logging. To disable logging, set the `verbose_logging` argument to `False`. Note that float values in the logs are rounded to 2 decimal points.
[INFO 12-27 05:08:29] ax.service.utils.instantiation: Inferred value type of ParameterType.INT for parameter accum_iter. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 12-27 05:08:29] ax.service.utils.instantiation: Inferred value type of ParameterType.INT for parameter batch_size. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'b

== Status ==
Memory usage on this node: 5.3/31.3 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 8.000: None | Iter 4.000: None | Iter 2.000: None
Resources requested: 8/8 CPUs, 1/1 GPUs, 0.0/16.16 GiB heap, 0.0/5.57 GiB objects (0/1.0 accelerator_type:RTX)
Result logdir: /opt/favordata/AI/Felix/kaggle-cassava/ray-results/ax3
Number of trials: 1/50 (1 RUNNING)
+------------------+----------+-------+--------------+--------------+--------------+-----------+-------------+------------------+----------------+
| Trial name       | status   | loc   |   accum_iter |   batch_size | is_amsgrad   |        lr | middle_fc   |   middle_fc_size |   weight_decay |
|------------------+----------+-------+--------------+--------------+--------------+-----------+-------------+------------------+----------------|
| DEFAULT_8f195b20 | RUNNING  |       |            2 |            8 | False        | 0.0072073 | False       |                0 |         0.0005 |
+------------------+----------+-------+----

Result for DEFAULT_8f195b20:
  accuracy: 0.5532
  date: 2020-12-27_05-34-53
  done: false
  experiment_id: 0d97abcc07a441c2aaaf8b522555c829
  experiment_tag: 1_accum_iter=2,batch_size=8,is_amsgrad=False,lr=0.0072073,middle_fc=False,middle_fc_size=0,weight_decay=0.0005
  hostname: Nevsky
  iterations_since_restore: 4
  loss: 1.2628076981894578
  node_ip: 10.0.0.200
  pid: 3301
  should_checkpoint: true
  time_since_restore: 1582.29887509346
  time_this_iter_s: 395.3860414028168
  time_total_s: 1582.29887509346
  timestamp: 1609047293
  timesteps_since_restore: 0
  training_iteration: 4
  trial_id: 8f195b20
  
== Status ==
Memory usage on this node: 7.6/31.3 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 8.000: None | Iter 4.000: -1.2628076981894578 | Iter 2.000: -1.243230769905863
Resources requested: 8/8 CPUs, 1/1 GPUs, 0.0/16.16 GiB heap, 0.0/5.57 GiB objects (0/1.0 accelerator_type:RTX)
Result logdir: /opt/favordata/AI/Felix/kaggle-cassava/ray-results/ax3
Number of trials: 2/5

(pid=3301) Epoch     8: adjusting learning rate of group 0 to 6.8914e-04.
Result for DEFAULT_8f195b20:
  accuracy: 0.564
  date: 2020-12-27_06-01-13
  done: false
  experiment_id: 0d97abcc07a441c2aaaf8b522555c829
  experiment_tag: 1_accum_iter=2,batch_size=8,is_amsgrad=False,lr=0.0072073,middle_fc=False,middle_fc_size=0,weight_decay=0.0005
  hostname: Nevsky
  iterations_since_restore: 8
  loss: 1.2458355705949324
  node_ip: 10.0.0.200
  pid: 3301
  should_checkpoint: true
  time_since_restore: 3162.5517632961273
  time_this_iter_s: 395.01573634147644
  time_total_s: 3162.5517632961273
  timestamp: 1609048873
  timesteps_since_restore: 0
  training_iteration: 8
  trial_id: 8f195b20
  
== Status ==
Memory usage on this node: 7.6/31.3 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 8.000: -1.2458355705949324 | Iter 4.000: -1.2628076981894578 | Iter 2.000: -1.243230769905863
Resources requested: 8/8 CPUs, 1/1 GPUs, 0.0/16.16 GiB heap, 0.0/5.57 GiB objects (0/1.0 accelerator_type:RTX

[INFO 12-27 06:14:23] ax.service.ax_client: Completed trial 0 with data: {'loss': (1.22, 0.0)}.
[INFO 12-27 06:14:23] ax.service.ax_client: Generated new trial 2 with parameters {'lr': 0.0, 'accum_iter': 4, 'batch_size': 8, 'weight_decay': 0.0, 'is_amsgrad': False, 'middle_fc': False, 'middle_fc_size': 0}.


...

== Status ==
Memory usage on this node: 5.1/31.3 GiB
Using AsyncHyperBand: num_stopped=17
Bracket: Iter 8.000: -1.1242831896377514 | Iter 4.000: -1.1377810292606112 | Iter 2.000: -1.416282648904414
Resources requested: 0/8 CPUs, 0/1 GPUs, 0.0/16.16 GiB heap, 0.0/5.57 GiB objects (0/1.0 accelerator_type:RTX)
Result logdir: /opt/favordata/AI/Felix/kaggle-cassava/ray-results/ax3
Number of trials: 46/50 (28 ERROR, 1 PENDING, 17 TERMINATED)
+------------------+------------+-------+--------------+--------------+--------------+------------+-------------+------------------+----------------+-------------+------------+----------------------+
| Trial name       | status     | loc   |   accum_iter |   batch_size | is_amsgrad   |         lr | middle_fc   |   middle_fc_size |   weight_decay |        loss |   accuracy |   training_iteration |
|------------------+------------+-------+--------------+--------------+--------------+------------+-------------+------------------+----------------+-----------

[INFO 12-27 13:43:57] ax.service.ax_client: Generated new trial 46 with parameters {'lr': 0.0, 'accum_iter': 8, 'batch_size': 8, 'weight_decay': 0.0, 'is_amsgrad': False, 'middle_fc': False, 'middle_fc_size': 0}.
2020-12-27 13:43:58,889	ERROR trial_runner.py:793 -- Trial DEFAULT_90ba54be: Error processing event.
Traceback (most recent call last):
  File "/opt/favordata/anaconda3/envs/kaggle/lib/python3.7/site-packages/ray/tune/trial_runner.py", line 726, in _process_trial
    result = self.trial_executor.fetch_result(trial)
  File "/opt/favordata/anaconda3/envs/kaggle/lib/python3.7/site-packages/ray/tune/ray_trial_executor.py", line 489, in fetch_result
    result = ray.get(trial_future[0], timeout=DEFAULT_GET_TIMEOUT)
  File "/opt/favordata/anaconda3/envs/kaggle/lib/python3.7/site-packages/ray/worker.py", line 1452, in get
    raise value.as_instanceof_cause()
ray.exceptions.RayTaskError(RuntimeError): ray::ImplicitFunc.train() (pid=23376, ip=10.0.0.200)
  File "python/ray/_raylet.pyx

2020-12-27 13:44:02,784	WARNING worker.py:1091 -- Failed to unpickle actor class 'ImplicitFunc' for actor ID 62d46b7801000000. Traceback:
Traceback (most recent call last):
  File "/opt/favordata/anaconda3/envs/kaggle/lib/python3.7/site-packages/ray/function_manager.py", line 493, in _load_actor_class_from_gcs
    actor_class = pickle.loads(pickled_class)
AttributeError: Can't get attribute 'setup' on <module 'common_utils' from '/opt/favordata/AI/Felix/kaggle-cassava/common_utils.py'>

(pid=23521) 2020-12-27 13:44:02,772	ERROR function_manager.py:495 -- Failed to load actor class ImplicitFunc.
(pid=23521) Traceback (most recent call last):
(pid=23521)   File "/opt/favordata/anaconda3/envs/kaggle/lib/python3.7/site-packages/ray/function_manager.py", line 493, in _load_actor_class_from_gcs
(pid=23521)     actor_class = pickle.loads(pickled_class)
(pid=23521) AttributeError: Can't get attribute 'setup' on <module 'common_utils' from '/opt/favordata/AI/Felix/kaggle-cassava/common_utils.py

== Status ==
Memory usage on this node: 5.0/31.3 GiB
Using AsyncHyperBand: num_stopped=17
Bracket: Iter 8.000: -1.1242831896377514 | Iter 4.000: -1.1377810292606112 | Iter 2.000: -1.416282648904414
Resources requested: 0/8 CPUs, 0/1 GPUs, 0.0/16.16 GiB heap, 0.0/5.57 GiB objects (0/1.0 accelerator_type:RTX)
Result logdir: /opt/favordata/AI/Felix/kaggle-cassava/ray-results/ax3
Number of trials: 50/50 (32 ERROR, 1 PENDING, 17 TERMINATED)
+------------------+------------+-------+--------------+--------------+--------------+------------+-------------+------------------+----------------+-------------+------------+----------------------+
| Trial name       | status     | loc   |   accum_iter |   batch_size | is_amsgrad   |         lr | middle_fc   |   middle_fc_size |   weight_decay |        loss |   accuracy |   training_iteration |
|------------------+------------+-------+--------------+--------------+--------------+------------+-------------+------------------+----------------+-----------

2020-12-27 13:44:04,132	ERROR trial_runner.py:793 -- Trial DEFAULT_93d398a4: Error processing event.
Traceback (most recent call last):
  File "/opt/favordata/anaconda3/envs/kaggle/lib/python3.7/site-packages/ray/tune/trial_runner.py", line 726, in _process_trial
    result = self.trial_executor.fetch_result(trial)
  File "/opt/favordata/anaconda3/envs/kaggle/lib/python3.7/site-packages/ray/tune/ray_trial_executor.py", line 489, in fetch_result
    result = ray.get(trial_future[0], timeout=DEFAULT_GET_TIMEOUT)
  File "/opt/favordata/anaconda3/envs/kaggle/lib/python3.7/site-packages/ray/worker.py", line 1452, in get
    raise value.as_instanceof_cause()
ray.exceptions.RayTaskError(RuntimeError): ray::ImplicitFunc.train() (pid=23543, ip=10.0.0.200)
  File "python/ray/_raylet.pyx", line 443, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 477, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 481, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx